### pre processing

In [1]:
#1
import pandas as pd
import math
import numpy as np
from numpy.lib.function_base import average
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras import models, layers
import tensorflow as tf

In [2]:
#2
import json
import os
import math
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import glob
from sklearn.metrics import confusion_matrix
import IPython.display as ipd  # To play sound in the notebook
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import os
import sys
import warnings
# ignore warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
import json
import os
import math
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from tensorflow.keras import models, layers
from scipy.io import wavfile
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate, Permute, Reshape
import os
import math
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM,TimeDistributed
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import glob
from sklearn.metrics import confusion_matrix
import IPython.display as ipd  # To play sound in the notebook
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import os
import sys
import warnings
# ignore warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
#from attention import Attention

In [3]:
#3
!pip install tensorflow



In [4]:

!git clone https://github.com/tensorflow/models.git



Cloning into 'models'...
remote: Enumerating objects: 93154, done.
remote: Counting objects: 100% (3044/3044), done.
remote: Compressing objects: 100% (1654/1654), done.
remote: Total 93154 (delta 1423), reused 2941 (delta 1366), pack-reused 90110
Receiving objects: 100% (93154/93154), 616.94 MiB | 17.74 MiB/s, done.
Resolving deltas: 100% (66360/66360), done.


In [5]:
cd models/research/audioset/vggish


/content/models/research/audioset/vggish


In [6]:
!wget https://storage.googleapis.com/audioset/vggish_model.ckpt
!wget https://storage.googleapis.com/audioset/vggish_pca_params.npz


--2024-02-27 15:38:27--  https://storage.googleapis.com/audioset/vggish_model.ckpt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.167.207, 172.253.115.207, 172.253.122.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.167.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 291108469 (278M) [application/octet-stream]
Saving to: ‘vggish_model.ckpt’

vggish_model.ckpt   100%[===================>] 277.62M  91.2MB/s    in 3.0s    

2024-02-27 15:38:31 (91.2 MB/s) - ‘vggish_model.ckpt’ saved [291108469/291108469]

--2024-02-27 15:38:31--  https://storage.googleapis.com/audioset/vggish_pca_params.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.167.207, 172.253.115.207, 172.253.122.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.167.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73020 (71K) [application/octet-stream]
Saving to: ‘vgg

In [7]:
!pip install resampy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.6 MB/s eta 0:00:00


In [8]:
import tensorflow as tf
import vggish_input
import vggish_params
import vggish_postprocess
import vggish_slim

In [9]:
# Disable eager execution
tf.compat.v1.disable_eager_execution()

In [10]:


tf.compat.v1.reset_default_graph()
sess = tf.compat.v1.Session()

# Load VGGish model
vggish_slim.define_vggish_slim(training=False)
vggish_slim.load_vggish_slim_checkpoint(sess, '/content/models/research/audioset/vggish/vggish_model.ckpt')


In [11]:
# Path to the folder containing audio clips
audio_folder = '/content/drive/MyDrive/MySelfVersionProjects/audio_sample/Sample_set_6_auto_labeling_validation/flac_sample_set6'
# Get a list of all audio files in the folder
audio_files = [f for f in os.listdir(audio_folder) if f.endswith('.flac')]

# Initialize lists to store VGGish features, paths, and file names
vggish_features_list = []
paths_list = []
file_names_list = []

# Process each audio file in the folder
for audio_file in audio_files:
    audio_path = os.path.join(audio_folder, audio_file)

    # Example of processing audio data
    example_wav_data = vggish_input.wavfile_to_examples(audio_path)

    # Get VGGish embeddings
    features_tensor = sess.graph.get_tensor_by_name(vggish_params.INPUT_TENSOR_NAME)
    embedding_tensor = sess.graph.get_tensor_by_name(vggish_params.OUTPUT_TENSOR_NAME)
    vggish_features = sess.run(embedding_tensor, feed_dict={features_tensor: example_wav_data})

    # Append the features, path, and file name to the respective lists
    vggish_features_list.append(vggish_features)
    paths_list.append(audio_path)
    file_names_list.append(audio_file)

# Create a DataFrame
df = pd.DataFrame({
    'Path': paths_list,
    'FileName': file_names_list
})

In [12]:
df

,Path,FileName
0,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2003817.flac
1,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2110511.flac
2,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2109047.flac
3,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2749149.flac
4,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2749668.flac
...,...,...
85,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2087130.flac
86,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2116995.flac
87,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2084586.flac
88,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2083681.flac


In [13]:
# Pad shorter clips with zeros to match the maximum number of frames
max_frames = max(features.shape[0] for features in vggish_features_list)
vggish_features_list_padded = [np.pad(features, ((0, max_frames - features.shape[0]), (0, 0)), mode='constant') for features in vggish_features_list]
vggish_features_array = np.stack(vggish_features_list_padded, axis=0)


In [14]:




# Save the NumPy array and DataFrame to files
np.save('/content/drive/MyDrive/MySelfVersionProjects/audio_sample/Sample_set_6_auto_labeling_validation/vggish_features_array_sample_set6_part2.npy', vggish_features_array)


# Continue with the rest of your code...


In [15]:
df.to_excel('/content/drive/MyDrive/MySelfVersionProjects/audio_sample/Sample_set_6_auto_labeling_validation/audio_file_info_sample_set6_part2.xlsx', index=False)

### autolabeling

In [16]:
from keras.models import load_model


In [17]:


# Later, to load the model:
loaded_model_audioQuality = load_model("/content/drive/MyDrive/MySelfVersionProjects/Codes/Sample_set_5_codes/EDLF_auto_labeling_with_appended_dataset/Saved Models/CNNmodel_audioQuality.h5")

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
loaded_model_pitch = load_model("/content/drive/MyDrive/MySelfVersionProjects/Codes/Sample_set_5_codes/EDLF_auto_labeling_with_appended_dataset/Saved Models/CNNmodel_PitchAnomaly.h5")

In [19]:
loaded_model_breath = load_model("/content/drive/MyDrive/MySelfVersionProjects/Codes/Sample_set_5_codes/EDLF_auto_labeling_with_appended_dataset/Saved Models/CNNmodel_breath.h5")

In [22]:
len(vggish_features_array[2][1])

128

In [23]:
import numpy as np

# Assuming arr1 is the array with shape (20, 11, 128) and arr2 is the array with shape (90, 8, 128)

arr2 = vggish_features_array    # Example array 2, replace with your data

# Define the target shape
target_shape = (10, 11, 128)

# Calculate the number of columns to pad
columns_to_pad = target_shape[1] - arr2.shape[1]

# Pad the array with zeros along the second axis to match the target shape
padded_arr2 = np.pad(arr2, ((0, 0), (0, columns_to_pad), (0, 0)), mode='constant', constant_values=0)

print("Original shape of arr2:", arr2.shape)
print("Padded shape of arr2:", padded_arr2.shape)


Original shape of arr2: (90, 8, 128)
Padded shape of arr2: (90, 11, 128)


In [24]:
padded_arr2_reshaped = padded_arr2.reshape((padded_arr2.shape[0], -1))

In [25]:
audio_quality_anomaly = loaded_model_audioQuality.predict(padded_arr2)

In [26]:
padded_arr2_flat = padded_arr2.flatten()

In [27]:
len(padded_arr2_flat)

126720

In [28]:
len(padded_arr2_reshaped[1])

1408

In [ ]:
padded_arr2_flat

array([ 0.91448903, -0.40318868,  0.5164564 , ...,  0.        ,
        0.        ,  0.        ], dtype=float32)

In [29]:
# CNN for TF expects (batch, height, width, channels)
# So we reshape the input tensors with a "color" channel of 1
padded_arr2_reshaped = padded_arr2_reshaped.reshape(padded_arr2_reshaped.shape[0],
                          padded_arr2_reshaped.shape[1],
                          1)

In [30]:
pitch_anomaly = loaded_model_pitch.predict(padded_arr2_reshaped)

In [31]:
breath = loaded_model_breath.predict(padded_arr2_reshaped)

In [32]:
audio_quality_anomaly

array([[0.8307463 ],
       [0.6015998 ],
       [0.5394858 ],
       [0.7639123 ],
       [0.7643625 ],
       [0.90562505],
       [0.77936226],
       [0.522833  ],
       [0.5234755 ],
       [0.49156162],
       [0.94668895],
       [0.74122745],
       [0.74476993],
       [0.7219084 ],
       [0.9455038 ],
       [0.8478347 ],
       [0.87333554],
       [0.5047695 ],
       [0.96728885],
       [0.49156162],
       [0.6991501 ],
       [0.9364064 ],
       [0.8919152 ],
       [0.8182246 ],
       [0.29052937],
       [0.72940564],
       [0.6173745 ],
       [0.93519676],
       [0.6639053 ],
       [0.7563651 ],
       [0.8422231 ],
       [0.9106892 ],
       [0.75190353],
       [0.971505  ],
       [0.53437704],
       [0.72426623],
       [0.5151833 ],
       [0.76690096],
       [0.39708352],
       [0.73681736],
       [0.7625976 ],
       [0.5606088 ],
       [0.822311  ],
       [0.70274085],
       [0.8884376 ],
       [0.5168846 ],
       [0.8690894 ],
       [0.745

In [33]:
df_labels = pd.DataFrame({"breath": breath.flatten(),
                    "pitch_anomaly": pitch_anomaly.flatten(),
                    "audio_quality_anomaly":audio_quality_anomaly.flatten()})

In [34]:
df_labels

,breath,pitch_anomaly,audio_quality_anomaly
0,0.001969,0.095105,0.830746
1,0.329447,0.398916,0.601600
2,0.136229,0.141490,0.539486
3,0.798660,0.069902,0.763912
4,0.000425,0.818002,0.764363
...,...,...,...
85,0.111213,0.584420,0.701106
86,0.011434,0.571164,0.568424
87,0.011950,0.634187,0.556079
88,0.797673,0.413443,0.635444


In [36]:
df_main = pd.merge(df, df_labels, right_index=True, left_index=True)

In [37]:
df_main

,Path,FileName,breath,pitch_anomaly,audio_quality_anomaly
0,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2003817.flac,0.001969,0.095105,0.830746
1,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2110511.flac,0.329447,0.398916,0.601600
2,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2109047.flac,0.136229,0.141490,0.539486
3,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2749149.flac,0.798660,0.069902,0.763912
4,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2749668.flac,0.000425,0.818002,0.764363
...,...,...,...,...,...
85,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2087130.flac,0.111213,0.584420,0.701106
86,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2116995.flac,0.011434,0.571164,0.568424
87,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2084586.flac,0.011950,0.634187,0.556079
88,/content/drive/MyDrive/MySelfVersionProjects/a...,DF_E_2083681.flac,0.797673,0.413443,0.635444


In [38]:
df_main.to_excel('/content/drive/MyDrive/MySelfVersionProjects/audio_sample/Sample_set_6_auto_labeling_validation/labels_part2.xlsx', index=False)

In [47]:
df_manual = pd.read_excel("/content/drive/MyDrive/MySelfVersionProjects/audio_sample/Sample_set_6_auto_labeling_validation/labels_sample_set6_part2_manual.xlsx")

In [48]:
df_manual

,link,Path,FileName,breath,pitch_anomaly,audio_quality_anomaly,Chloe's notes,Breath (CE),Pitch (CE),SoundQuality (CE),Christine's Notes,Breath (CM),Pitch (CM),SoundQuality (CM)
0,https://drive.google.com/file/d/1o-MICG4ZUmZRY...,NaN,DF_E_2000026.flac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://drive.google.com/file/d/1BSwFLVPZNc0aS...,NaN,DF_E_2000053.flac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://drive.google.com/file/d/1wZyDljWO9YEuQ...,NaN,DF_E_2000211.flac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://drive.google.com/file/d/1Os9rAQbcRWpe_...,NaN,DF_E_2000528.flac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://drive.google.com/file/d/1jFO7SjeQBOP5w...,NaN,DF_E_2000692.flac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,https://drive.google.com/file/d/1spgzoktV9xyL-...,NaN,DF_E_2746882.flac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,https://drive.google.com/file/d/1bw3Fluowq1gcm...,NaN,DF_E_2748035.flac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,https://drive.google.com/file/d/1BoqIO4iSz1Dq8...,NaN,DF_E_2748680.flac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,https://drive.google.com/file/d/13xatQjI4et4US...,NaN,DF_E_2749149.flac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
df_main2 = pd.merge( df_main, df_manual, on="FileName")

In [51]:
df_main2.to_excel("/content/drive/MyDrive/MySelfVersionProjects/audio_sample/Sample_set_6_auto_labeling_validation/labels_part2_completed_sample_set6.xlsx")